<a href="https://colab.research.google.com/github/LaylaMcD/DIGS-DP/blob/main/Word2Vec_Pt_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import pandas
import gensim
import logging
import glob, os
import nltk
from nltk import word_tokenize
from pprint import pprint
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [20]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [22]:
# directory containing all source texts for training the model
data_dir="/content/drive/My Drive/Colab Notebooks/output_files"
os.chdir(data_dir)

data = []
for filename in glob.glob("*.txt"):
    filedata = open(filename, 'r').read()
    data.append(filedata)

print("Number of documents: " + str(len(data)))

Number of documents: 10


In [23]:
import nltk

nltk.download("popular")

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

True

In [25]:
from nltk.corpus import wordnet as wn
from gensim import corpora, models
#from contractions import CONTRACTION_MAP
import re, string
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()

def tokenize_text(text):
    tokens = nltk.word_tokenize(text)
    tokens = [token.strip() for token in tokens]
    return tokens

def pos_tag_text(text):
    # convert Penn treebank tag to wordnet tag
    def penn_to_wn_tags(pos_tag):
        if pos_tag.startswith('J'):
            return wn.ADJ
        elif pos_tag.startswith('V'):
            return wn.VERB
        elif pos_tag.startswith('N'):
            return wn.NOUN
        elif pos_tag.startswith('R'):
            return wn.ADV
        else:
            return None
    tagged_text = nltk.pos_tag(nltk.word_tokenize(text))
    #tagged_text = tag(text)
    tagged_lower_text = [(word.lower(), penn_to_wn_tags(pos_tag)) for word, pos_tag in tagged_text]
    return tagged_lower_text

def lemmatize_text(text):
    pos_tagged_text = pos_tag_text(text)
    lemmatized_tokens = [wnl.lemmatize(word, pos_tag) if pos_tag else word for word, pos_tag in pos_tagged_text]
    lemmatized_text = ' '.join(lemmatized_tokens)
    return lemmatized_text

def remove_special_characters(text):
    tokens = tokenize_text(text)
    pattern = re.compile('[{}]'.format(re.escape(string.punctuation)))
    filtered_tokens = filter(None, [pattern.sub('', token) for token in tokens])
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

def remove_stopwords(text):
    stopword_list = nltk.corpus.stopwords.words('english')
    tokens = tokenize_text(text)
    filtered_tokens = [token for token in tokens if token not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

def normalize_corpus(corpus, tokenize):
    normalized_corpus = []
    for text in corpus:
        text = lemmatize_text(text)
        text = remove_special_characters(text)
        text = remove_stopwords(text)
        if tokenize:
            text = tokenize_text(text)
        normalized_corpus.append(text)
    return normalized_corpus

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [26]:
norm_corpus = normalize_corpus(data, tokenize=True)

dictionary = Dictionary(norm_corpus)   # Build the dictionary

# Convert to vector corpus
vectors = [dictionary.doc2bow(text) for text in norm_corpus]

# Build TF-IDF model
tfidf = TfidfModel(vectors)

In [27]:
# Get TF-IDF weights for 50 most document-specific terms
weights = tfidf[vectors[0]]

# Get terms from the dictionary and pair with weights
weights = [(dictionary[pair[0]], pair[1]) for pair in weights]

pprint(sorted(weights, key=lambda weights: weights[1], reverse=True)[1:50])

[('snbts', 0.37114033360913157),
 ('haemophilia', 0.3057580986728079),
 ('liberton', 0.2346068430068086),
 ('z8', 0.16537859425070114),
 ('ludlam', 0.1384564975122149),
 ('berühmte', 0.136533490602323),
 ('basler', 0.12884146296275553),
 ('ewp', 0.10384237313416117),
 ('cryoprecipitate', 0.09812106281460813),
 ('dpc', 0.0942273385847018),
 ('swara', 0.09230433167480993),
 ('warmest', 0.08461230403524243),
 ('fviii', 0.07884328330556681),
 ('lta', 0.07692027639567493),
 ('wettest', 0.07692027639567493),
 ('d800', 0.0725827039998471),
 ('sanskaar', 0.07115125566599932),
 ('ihre', 0.06720620740726584),
 ('19712000', 0.0653822349363237),
 ('egfr', 0.0653822349363237),
 ('shhd', 0.06345922802643184),
 ('8y', 0.05961321420664808),
 ('buckalew', 0.05576720038686433),
 ('wonmunna', 0.05576720038686433),
 ('cistern', 0.055109090073957985),
 ('transfusion', 0.054296969771322826),
 ('ius', 0.051921186567080585),
 ('ravitch', 0.04999817965718872),
 ('viii', 0.049385035532834014),
 ('mayne', 0.0480

In [28]:
def word_freq(search_term):
    lemmacounts = []
    for filename in glob.glob("*.txt"):
        filedata = open(filename, 'r').read()
        tokens = word_tokenize(filedata)
        lemmas = [nltk.stem.WordNetLemmatizer().lemmatize(token.lower()) for token in tokens]
        lemmacounts.append([filename, lemmas.count(search_term)])
    return lemmacounts

word_freq("interpol")

[['chunk_2.txt', 32],
 ['chunk_10.txt', 0],
 ['chunk_5.txt', 0],
 ['chunk_9.txt', 0],
 ['chunk_6.txt', 0],
 ['chunk_4.txt', 0],
 ['chunk_3.txt', 0],
 ['chunk_8.txt', 6],
 ['chunk_7.txt', 0],
 ['chunk_1.txt', 2]]

Looks like Interpol appeared a couple of times! Let's find the text

In [29]:
def freq_check(filename, search_term):
    filedata = open(filename, 'r').read()
    tokens = word_tokenize(filedata)
    lemmas = [nltk.stem.WordNetLemmatizer().lemmatize(token.lower()) for token in tokens]
    resultstr = "In file " + filename + ", the term '" + search_term + "' exists " + str(lemmas.count(search_term)) + " times" + \
    " compared to " + str(len(lemmas)) + " total lemmatized tokens for a raw frequency of " + str((lemmas.count(search_term) / len(lemmas)))
    return resultstr

freq_check('chunk_2.txt','interpol')

"In file chunk_2.txt, the term 'interpol' exists 32 times compared to 567929 total lemmatized tokens for a raw frequency of 5.634507130292695e-05"

What about in the other file?

In [30]:
def freq_check(filename, search_term):
    filedata = open(filename, 'r').read()
    tokens = word_tokenize(filedata)
    lemmas = [nltk.stem.WordNetLemmatizer().lemmatize(token.lower()) for token in tokens]
    resultstr = "In file " + filename + ", the term '" + search_term + "' exists " + str(lemmas.count(search_term)) + " times" + \
    " compared to " + str(len(lemmas)) + " total lemmatized tokens for a raw frequency of " + str((lemmas.count(search_term) / len(lemmas)))
    return resultstr

freq_check('chunk_1.txt','interpol')

"In file chunk_1.txt, the term 'interpol' exists 2 times compared to 547212 total lemmatized tokens for a raw frequency of 3.654890609124069e-06"

Now let's compare that word to the appearance of other words in the text files

In [31]:
# As a final check, provide the most frequent lemmas for that document (=file)
from pprint import pprint
from nltk import word_tokenize
from nltk.corpus import stopwords
import string

# directory containing all source texts for training the model
filename="chunk_1.txt"

filedata = open(filename, 'r').read()
xtokens = word_tokenize(filedata)
xlemmas = [nltk.stem.WordNetLemmatizer().lemmatize(token.lower()) for token in xtokens]

# Stopwords and punctuation removal
stop = stopwords.words('english') + list(string.punctuation)

tokens_nsp = [i.lower() for i in xtokens if i.lower() not in stop]
lemmas_nsp = [i.lower() for i in xlemmas if i.lower() not in stop]

##

print ("\nTokens:")

wfreq = []
for w in set(tokens_nsp):
    wfreq.append([w, tokens_nsp.count(w)])
wfreq_sorted = sorted(wfreq, key=lambda wfreq: wfreq[1], reverse=True)
pprint(wfreq_sorted[0:50])

print ("\nLemmas:")

lfreq = []
for w in set(lemmas_nsp):
    lfreq.append([w, lemmas_nsp.count(w)])
lfreq_sorted = sorted(lfreq, key=lambda lfreq: lfreq[1], reverse=True)
pprint(lfreq_sorted[0:50])


Tokens:
[["''", 4215],
 ['’', 3943],
 ['coloring', 2482],
 ['pages', 1817],
 ['``', 1584],
 ["'s", 1473],
 ['”', 1275],
 ['“', 1255],
 ['size', 1120],
 ['one', 1070],
 ['printable', 1039],
 ['like', 884],
 ['free', 837],
 ['time', 810],
 ['also', 806],
 ['new', 794],
 ['would', 775],
 ['get', 747],
 ["n't", 731],
 ['people', 657],
 ['said', 647],
 ['page', 615],
 ['first', 581],
 ['see', 579],
 ['may', 537],
 ['--', 525],
 ['make', 520],
 ['kids', 510],
 ['know', 503],
 ['best', 501],
 ['book', 488],
 ['well', 474],
 ['good', 465],
 ['day', 462],
 ['use', 446],
 ['year', 445],
 ['...', 438],
 ['two', 433],
 ['–', 428],
 ['even', 425],
 ['world', 419],
 ['us', 415],
 ['way', 413],
 ['many', 409],
 ['work', 409],
 ['1', 405],
 ['2', 401],
 ['great', 395],
 ['could', 394],
 ['love', 393]]

Lemmas:
[["''", 4215],
 ['’', 3943],
 ['coloring', 2482],
 ['page', 2432],
 ['wa', 2218],
 ['``', 1584],
 ["'s", 1473],
 ['”', 1275],
 ['“', 1255],
 ['ha', 1173],
 ['size', 1136],
 ['one', 1130],
 ['pr